In [6]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
base_dir = Path().resolve().parent
data_dir = base_dir / 'data'

In [17]:
def convertToFloat(inputString):
    result = ""
    for symbol in inputString:
        if symbol in "0123456789.,":
            result += symbol
    return float(result)

def createDF():
    f = open(data_dir / 'important_dates', "r")
    file = f.readlines()
    df = pd.DataFrame(columns=['event', "date", "year", 'month', "actual", 'forecastCorrection'])
    currentDate = ''
    days = ["Monday",
            "Tuesday",
            "Wednesday",
            "Thursday",
            "Friday",
            "Saturday",
            "Sunday"]

    months = [
         'APR',
         'AUG',
         'DEC',
         'FEB',
         'JAN',
         'JUL',
         'JUN',
         'MAR',
         'MAY',
         'NOV',
         'OCT',
         'SEP'
    ]

    Qs = [
        "Q1",
        "Q2",
        "Q3",
        "Q4"
    ]

    blockList = [
        "Interest Rate Decision",
        "Current Account",
        "EU",
        "Prel",
        "European Council",
        "Monetary Policy Report",
        "Bond Auction",
        "Bill Auction",
        "EcoFin",
        "Conference",
        "Commission",
        "Full Year GDP Growth",
        "Summit",
        "Election",
        "Meeting",
        "Debate",
        "Flash",
        "Car",
        "Vehicle",
        "PPI YoY",
        "GDP YoY",
        "Inflation Rate YoY",
        "M2 Money Supply YoY",
        "Markit Composite PMI"
    ]
    
    for line in file:
        flag = False

        # Пропускаем строчки со временем
        # Например, "04:00 timeFlag"
        if "timeFlag" in line:
            continue

        # Убираем флаг страны
        # Например, "RU"
        if line == "RU\n":
            continue

        # Вычленияем даты
        # У каждой даты первое слово - день недели на английском
        for day in days:
            if day in line:
                currentDate = line
                #print(f"DATE: {pd.to_datetime(currentDate)}")
                flag = True
        if flag:
            continue

        for block in blockList:
            if block in line:
                flag = True

        if flag:
            continue

        for month in months:
            if month in line:
                flag = True

                formalDate = pd.to_datetime(currentDate)
                numericData = line[line.index(month)+3:].strip().split("	")
                event = line[:line.index(month)]
                actual = convertToFloat(numericData[0])
                forecast = convertToFloat(numericData[-1])
                targetYear = formalDate.year
                if formalDate.month < 4 and month in ["DEC", "NOV", "OCT", "SEPT"]:
                    targetYear -= 1
              #  print(formalDate, line)

                forecastCorrection = forecastCorrection = actual - forecast

                df = df.append({'event': event, "date": formalDate, "year": targetYear, "month": month, "actual": actual, "forecastCorrection": forecastCorrection}, ignore_index=True)
                #print(line[:line.index(month)])
                #print(f"ACTUAL MONTH:{month} - {line}", end="")

        for Q in Qs:
            if Q in line:
                flag = True
                #print(f"ACTUAL QUARTER: {Q} {line}")

        if flag:
            continue
    return df

In [22]:
# Исправить разметку df
df = createDF()
from calendar import month_abbr
lower_ma = [m.lower() for m in month_abbr]
df['month'] = df['month'].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')
df["event"] = df["event"].str.strip()
df["event"][df.event.str.contains("Inflation Rate MoM")] = "Inflation Rate MoM"

C:\Users\un\AppData\Local\Temp/ipykernel_17020/578007669.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["event"][df.event.str.contains("Inflation Rate MoM")] = "Inflation Rate MoM"


In [24]:
df

,event,date,year,month,actual,forecastCorrection
0,Inflation Rate MoM,2017-01-10,2016,12,0.4,0.0
1,Balance of Trade,2017-01-16,2016,11,9.1,2.3
2,Foreign Exchange Reserves,2017-01-16,2016,12,377.7,2.6
3,Industrial Production YoY,2017-01-23,2016,12,3.2,1.0
4,Unemployment Rate,2017-01-25,2016,12,5.3,-0.2
...,...,...,...,...,...,...
714,Unemployment Rate,2021-12-29,2021,11,4.3,0.0
715,Real Wage Growth YoY,2021-12-29,2021,10,0.6,-0.4
716,Retail Sales YoY,2021-12-29,2021,11,3.1,-2.1
717,Business Confidence,2021-12-29,2021,12,3.3,0.3


In [25]:
tempResult = pd.read_excel(data_dir / "features_excel.xlsx")
tempResult.drop("Unnamed: 0", axis=1, inplace=True)
tempResult["Date"] = pd.to_datetime(tempResult["Date"])
tempResult

,Date,Income,Outcome,Balance,Налог на прибыль (при уплате только ежеквартальных авансовых платежей),Налог на прибыль (при уплате ежемесячных авансовых платежей с доплатой по итогам квартала),Налог на прибыль (при ежемесячной уплате авансов исходя из фактической прибыли) (АНАЛОГ F),НДС,Налог при УСН,НДФЛ с отпускных и больничных пособий\t,...,ЕСХН,Торговый сбор на территории г. Москвы,"Страховые взносы, перечисляемые в ИФНС","Взносы, уплачиваемые ИП за себя","Взносы, перечисляемые в ФСС",Налог на добычу полезных ископаемых (Последняя неделя каждого месяца),Зарплаты и авансы (не позднее 15 и 30 числа каждого месяца),Праздники РФ,COVID cases (см вкладку),GDP
0,2017-01-09,1.340000,1.490000,-0.155904,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.5
1,2017-01-10,1.068610,1.194182,-0.125572,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5
2,2017-01-11,0.944429,0.936663,0.007767,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5
3,2017-01-12,1.670000,0.875379,0.800391,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5
4,2017-01-13,0.955924,0.975645,-0.019721,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,2021-03-27,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1539,2021-03-28,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1540,2021-03-29,2.360000,3.370000,-1.017679,1.0,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1541,2021-03-30,2.320000,2.570000,-0.259687,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
import warnings
warnings.filterwarnings('ignore')

def getData(currentDate, currentType, currentEvent):
    return df[df.event.str.contains(currentEvent)][df.year==currentDate.year][df.month==currentDate.month].iloc[0][currentType]

for event in df.event.unique():
    for dataType in ["actual", "forecastCorrection"]:
        columnName = f"{event}: {dataType}"
        tempResult[columnName] = tempResult.apply(lambda row: getData(row.Date, dataType, event), axis = 1) 

In [27]:
tempResult.isna().sum()

Date                                                                                             0
Income                                                                                           0
Outcome                                                                                          0
Balance                                                                                          0
Налог на прибыль (при уплате только ежеквартальных авансовых платежей)                        1527
Налог на прибыль (при уплате ежемесячных авансовых платежей с доплатой по итогам квартала)    1492
Налог на прибыль (при ежемесячной уплате авансов исходя из фактической прибыли) (АНАЛОГ F)    1494
НДС                                                                                           1496
Налог при УСН                                                                                 1525
НДФЛ с отпускных и больничных пособий\t                                                       1492
ЕНВД      

In [30]:
tempResult.to_csv(data_dir / "featuresWithKeyFactors.csv")